# Data retrival and preprocessing

In [1]:
ticker = 'TATAMOTORS.NS'
num_of_days = 4

In [2]:
from models.MLPModel import MLPModel

model = MLPModel(ticker, num_of_days)


Using TensorFlow backend.


[*********************100%***********************]  1 of 1 downloaded
Instructions for updating:
Colocations handled automatically by placer.


In [3]:
model.normalized_data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-01-01,0.657293,0.691786,0.655377,0.690637,0.156121,116802.85
1997-01-02,0.695619,0.695619,0.676456,0.683642,0.154540,128507.66
1997-01-03,0.676456,0.689774,0.674827,0.687092,0.155320,79989.83
1997-01-06,0.680289,0.690349,0.672815,0.677414,0.153132,56131.43
1997-01-07,0.675977,0.675977,0.653939,0.659113,0.148995,114660.71
1997-01-08,0.669749,0.684121,0.665437,0.670036,0.151464,52462.92
1997-01-09,0.670036,0.675498,0.663042,0.667928,0.150987,56314.08
1997-01-10,0.674540,0.689678,0.672815,0.685175,0.154886,97257.43
1997-01-13,0.680289,0.680289,0.669366,0.676456,0.152915,42305.35


In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 4, 512)            2560      
_________________________________________________________________
dense_2 (Dense)              (None, 4, 512)            262656    
_________________________________________________________________
dense_3 (Dense)              (None, 4, 512)            262656    
_________________________________________________________________
dense_4 (Dense)              (None, 4, 512)            262656    
_________________________________________________________________
dense_5 (Dense)              (None, 4, 128)            65664     
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 513       
Total para

In [5]:
model.compile()

In [6]:
model.fit(500)

Instructions for updating:
Use tf.cast instead.
Train on 2076 samples, validate on 2077 samples
Epoch 1/500
2076/2076 [==============================] - 1s 694us/step - loss: 0.0080 - mean_absolute_percentage_error: 10.3582 - mean_absolute_error: 0.0427 - val_loss: 0.0052 - val_mean_absolute_percentage_error: 3.5721 - val_mean_absolute_error: 0.0530

Epoch 00001: val_loss improved from inf to 0.00515, saving model to TATAMOTORS.NS.weights.best.hdf5
Epoch 2/500
2076/2076 [==============================] - 1s 523us/step - loss: 5.2242e-04 - mean_absolute_percentage_error: 3.8909 - mean_absolute_error: 0.0161 - val_loss: 0.0036 - val_mean_absolute_percentage_error: 2.9716 - val_mean_absolute_error: 0.0439

Epoch 00002: val_loss improved from 0.00515 to 0.00360, saving model to TATAMOTORS.NS.weights.best.hdf5
Epoch 3/500
2076/2076 [==============================] - 1s 489us/step - loss: 4.2323e-04 - mean_absolute_percentage_error: 3.5784 - mean_absolute_error: 0.0146 - val_loss: 0.0041 - v


Epoch 00024: val_loss did not improve from 0.00218
Epoch 25/500
2076/2076 [==============================] - 1s 495us/step - loss: 3.1941e-04 - mean_absolute_percentage_error: 3.1381 - mean_absolute_error: 0.0129 - val_loss: 0.0023 - val_mean_absolute_percentage_error: 2.2983 - val_mean_absolute_error: 0.0345

Epoch 00025: val_loss did not improve from 0.00218
Epoch 26/500
2076/2076 [==============================] - 1s 488us/step - loss: 3.0553e-04 - mean_absolute_percentage_error: 3.0397 - mean_absolute_error: 0.0125 - val_loss: 0.0056 - val_mean_absolute_percentage_error: 3.6504 - val_mean_absolute_error: 0.0583

Epoch 00026: val_loss did not improve from 0.00218
Epoch 27/500
2076/2076 [==============================] - 1s 489us/step - loss: 3.4308e-04 - mean_absolute_percentage_error: 3.1897 - mean_absolute_error: 0.0132 - val_loss: 0.0063 - val_mean_absolute_percentage_error: 3.8351 - val_mean_absolute_error: 0.0632

Epoch 00027: val_loss did not improve from 0.00218
Epoch 28/500

2076/2076 [==============================] - 1s 467us/step - loss: 2.5996e-04 - mean_absolute_percentage_error: 2.8927 - mean_absolute_error: 0.0116 - val_loss: 0.0068 - val_mean_absolute_percentage_error: 3.9975 - val_mean_absolute_error: 0.0664

Epoch 00051: val_loss did not improve from 0.00209
Epoch 52/500
2076/2076 [==============================] - 1s 481us/step - loss: 4.3350e-04 - mean_absolute_percentage_error: 3.6595 - mean_absolute_error: 0.0152 - val_loss: 0.0029 - val_mean_absolute_percentage_error: 2.5191 - val_mean_absolute_error: 0.0396

Epoch 00052: val_loss did not improve from 0.00209
Epoch 53/500
2076/2076 [==============================] - 1s 487us/step - loss: 3.4402e-04 - mean_absolute_percentage_error: 3.2644 - mean_absolute_error: 0.0134 - val_loss: 0.0030 - val_mean_absolute_percentage_error: 2.6236 - val_mean_absolute_error: 0.0408

Epoch 00053: val_loss did not improve from 0.00209
Epoch 54/500
2076/2076 [==============================] - 1s 486us/step - los

In [7]:
model.load_weights()

In [8]:
score = model.evaluate()

1039/1039 [==============================] - 0s 99us/step


In [9]:
score

[0.008974134562137295, 1.6339636579858672, 0.0710503869224788]